# Ground Truth でシンプルな画像ラベリングタスクを実施する

このノートブックでは，Ground Truth を使った簡単なラベリングタスクを実際に行ってみることで，どのように Ground Truth を使うか理解することを目的とします．

## セットアップ

まずラベリングジョブを行うための IAM ポリシー，および データを作成します．


### ポリシーの準備

[IAM 管理ページ](https://console.aws.amazon.com/iam/home?region=us-east-1#/users) に移動して，ラベリングジョブを行うユーザーに対して，以下のポリシーを付与してください．この作業は，あらかじめハンズオンの前に管理者側で実施しておくことを推奨します．

- AmazonCognitoPowerUser ポリシー
- AWSLambdaFullAccess ポリシー

### データのセットアップ

データについては，以下のコマンドで処理対象のデータをダウンロードして，中身を確認します．

In [ ]:
!rm -rf data && aws s3 cp s3://makotosh-dataset/animal.zip ./data/ && unzip data/animal.zip -d data/

In [ ]:
% matplotlib inline 

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

im = Image.open('data/animal/cat/cat001.jpg', 'r')
plt.imshow(np.array(im))

続いて，ジョブを実行する際にファイル一覧を記述する，manifest.json ファイルを作成します．

In [ ]:
import os
import sagemaker

bucket = sagemaker_session.default_bucket()
pathes = []
for root, dirs, files in os.walk('data'):
        for file in files:
            if file.endswith('jpg'):
                pathes.append(os.path.join('s3://', bucket, JOB_NAME, root , file))

with open('data/animal/manifest.json', 'w') as f:
    for path in pathes:
        line = '{{"source-ref": "{path}"}}\n'.format(path=path) 
        f.write(line)

In [ ]:
!cat data/animal/manifest.json

### データのアップロード

ラベリングジョブの入力データをアップロードします．`JOB_NAME` の `XXX` を，`001` のように，ハンズオン実施時に指定された番号に置き換えてから，コマンドを実行してください．

In [ ]:
PATH = 'data/animal'
JOB_NAME = 'test-job-XXX'

sagemaker_session = sagemaker.Session()
sagemaker_session.upload_data(
        path=PATH,
        key_prefix=os.path.join(JOB_NAME, PATH))

### ラベリングジョブを行うチームの作成

[SageMaker コンソールのラベリング労働力ページ](https://console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/labeling-workforces) に移動して，上のメニューから `Private` を選択し，`Create private team` を押してください．続いて以下のように入力したら，Create private team を押します．

- Team name: `private-team-XXX` と入力してください．XXX は先ほどと同様に，ハンズオン開始時に指定された番号になります
- Add workers: `Create a new Amazon Cognito user group` を選択してください
- Email addresses: 自身のメールアドレスを入力してください
- Organization name: `private-org-XXX`

元の画面に戻ったら，Private teams 内に，作成したチームが表示されています（表示されていない場合は，画面をリロードしてください）．しばらくすると，入力したメールアドレスに，Amazon SageMaker から招待リンクの含まれたメールが届きます．そうしたら，リンクをクリックして，メールに書かれたユーザー名とパスワードを使ってログインし，パスワードを変更します．これで準備ができました．

## ラベリングジョブの作成

まず，以下のコマンドを実行して，必要な値を生成します．

In [ ]:
print('Job name: {}'.format(JOB_NAME))
print('Input data location: s3://{}/{}/{}/manifest.json'.format(
    sagemaker_session.default_bucket(), JOB_NAME, PATH))
print('Output data location: s3://{}'.format(sagemaker_session.default_bucket()))

[SageMaker コンソールのラベリングジョブページ](https://console.aws.amazon.com/sagemaker/groundtruth?region=us-east-1#/labeling-jobs) に移動して，`Create labelling job ` ボタンをクリックします．その後以下のように入力を行ってから，「Next」ボタンを押してください．

- Job name: 上の値をそのままコピー
- Input dataset location: 上の値をそのままコピー
- Output dataset location: 上の値をそのままコピー
- IAM role: 既存の SageMakerFullAccess ポリシーを付与したロールを使用

![labelling_image](./img/img002.jpg)

その上で，Task type として Image classification を選択したら Next を押してください．

次のページでは，順番に以下のように選択してください．

- Worker types: Private を選択
- Private teams: 先ほど作成したチームを選択

![labelling_image](./img/img003.jpg)

その上で，Image classification labelling tool について，以下のように追加情報の入力を行います．

- Enter a brief description of the task: タスクに関する説明文を各フィールドです．ここでは「画像分類のテスト」と入力してください
- Select an option: 2 つのフィールドに対して「dog」「cat」とそれぞれ入力してください．このフィールドには日本語は使用できません

![labelling_image](./img/img004.jpg)

これらが終わったら Submit ボタンを押してください．

## ラベリングジョブの実施

あとは，先ほどの Worker 側作業ページを開いて待っていると，分類タスクが積まれます．実際にラベリングタスクを試してみてください．

![labelling_image](./img/img004.jpg)

## 結果の確認

ジョブが終了したら，マネジメントコンソール内のラベリングジョブの一覧から，実行したジョブを選択して，Output dataset location をクリックしてください．アウトプットフォルダは，以下のような階層構造になっています．

- s3://PATH/TO/JOB/OUTPUT/activelearning: 自動ラベリング機能を使った時に，その結果を格納するディレクトリです
- s3://PATH/TO/JOB/OUTPUT/annotations: 人手によるラベリングジョブの結果が格納されるディレクトリです．個々人のラベリング結果を入れる worker-responce サブディレクトリ，各バッチごとのアノテーション結果をまとめた consolidated-annotation サブディレクトリ，バッチ内の判定結果をマニフェストファイルの形に落とした intermediate サブディレクトリの 3 つが含まれます
- s3://PATH/TO/JOB/OUTPUT/inference: 自動ラベリング機能の実施時の，バッチ推論ジョブの入出力データが格納されるディレクトリです
- s3://PATH/TO/JOB/OUTPUT/manifests: 最終結果のマニフェストファイルが格納されるディレクトリです
- s3://PATH/TO/JOB/OUTPUT/training: 自動ラベリング機能の実施時の，学習ジョブに関するデータが格納されるディレクトリです

上記以外の詳細な説明は[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-data-output.html)をご覧ください．

では，最終的なラベリング結果を取得して中身を確認しましょう．下記コマンドを実行して，中身を確認します．

In [ ]:
import sys

s3.download_file(BUCKET_NAME, '{}/manifests/output/output.manifest'.format(JOB_NAME), 'output.manifest')

with open('output.manifest', 'r') as f:
    for line in f.readlines():
        print(line)